In [1]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas
from tqdm import tqdm

In [2]:
dataset = load_dataset("presencesw/vinli_3_label")

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 18282
    })
    test: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 2264
    })
    dev: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 2255
    })
})


In [4]:
def convert_entailment(exmaples):
    labels = exmaples['gold_label']
    labels = ["not_entailment" if i != 'entailment' else 'entailment' for i in labels]
    exmaples['gold_label'] = labels
    return exmaples

In [5]:
new_dataset = dataset.map(convert_entailment, batched=True)

In [6]:
temp = Dataset.from_dict(new_dataset['train'][:10])
for i in temp:
    print(i)

{'gold_label': 'entailment', 'sentence1': 'Sau dự án điện mặt trời, Tập đoàn Trung Nam vừa bán 35,1% cổ phần Nhà máy Điện gió Trung Nam cho Công ty Hitachi Sustainable Energy.', 'sentence2': 'Công ty Hitachi Sustainable Energy có cổ phần trong Nhà máy Điện gió Trung Nam.'}
{'gold_label': 'entailment', 'sentence1': 'Sau dự án điện mặt trời, Tập đoàn Trung Nam vừa bán 35,1% cổ phần Nhà máy Điện gió Trung Nam cho Công ty Hitachi Sustainable Energy.', 'sentence2': 'Tập đoàn Trung Nam vừa thực hiện dự án điện năng lượng mặt trời.'}
{'gold_label': 'not_entailment', 'sentence1': 'Sau dự án điện mặt trời, Tập đoàn Trung Nam vừa bán 35,1% cổ phần Nhà máy Điện gió Trung Nam cho Công ty Hitachi Sustainable Energy.', 'sentence2': 'Tập đoàn Trung Nguyên vừa bán hơn hơn 30% cổ phần cho Tập đoàn Vingroup.'}
{'gold_label': 'not_entailment', 'sentence1': 'Sau dự án điện mặt trời, Tập đoàn Trung Nam vừa bán 35,1% cổ phần Nhà máy Điện gió Trung Nam cho Công ty Hitachi Sustainable Energy.', 'sentence2': '

In [7]:
new_dataset['train']

Dataset({
    features: ['gold_label', 'sentence1', 'sentence2'],
    num_rows: 18282
})

In [8]:
new_dataset_entail = new_dataset.filter(lambda example: example['gold_label'] == 'entailment')['train']
new_dataset_not_entail = new_dataset.filter(lambda example: example['gold_label'] == 'not_entailment')['train']

In [9]:
new_dataset_entail = Dataset.to_pandas(new_dataset_entail)
new_dataset_not_entail1 = Dataset.to_pandas(new_dataset_not_entail)
new_dataset_not_entail2 = Dataset.to_pandas(new_dataset_not_entail)

In [10]:
j = 0
for i in tqdm(range(len(new_dataset_not_entail1))):
    temp = set()
    # check = False
    # for j in range(len(new_dataset_not_entail2)):
    for index in range(100):
        new_j = j + index
        if new_j == len(new_dataset_not_entail1):
            break
        if new_dataset_not_entail1.iloc[i]['sentence1'] != new_dataset_not_entail2.iloc[new_j]['sentence1']:
            j = new_j
            break
        temp.add(new_dataset_not_entail2.iloc[new_j]['sentence2'])
    new_dataset_not_entail1.iloc[i]['sentence2'] = temp

100%|██████████| 12188/12188 [00:02<00:00, 5105.04it/s]


In [11]:
print(len(new_dataset_not_entail1))

12188


In [12]:
# print(new_dataset_not_entail1.iloc[2]['sentence2'])

In [13]:
dataset_not_entail = new_dataset_not_entail1.drop_duplicates(subset=['sentence2'])
print(len(dataset_not_entail))
dataset_not_entail = dataset_not_entail[dataset_not_entail['sentence2'] != set()]
print(len(dataset_not_entail))

3045
3044


In [14]:
dataset_not_entail = dataset_not_entail.reset_index(drop=True)

In [15]:
dataset_not_entail

,gold_label,sentence1,sentence2
0,not_entailment,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",{Nhà máy Điện gió Trung Nam có 100% vốn cổ phầ...
1,not_entailment,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",{Nhiều người mong muốn được giữ quyền điều hàn...
2,not_entailment,"Cách đây một tháng, tập đoàn này cũng bán 49% ...",{Vốn của một công ty nước ngoài đang chiếm 49%...
3,not_entailment,"Dự án Nhà máy Điện gió Trung Nam, tổng số vốn ...",{Sản lượng điện dự kiến tạo ra trong một năm t...
4,not_entailment,"Từ nay tới năm 2028, dòng vốn đầu tư được dự b...",{Theo như dự đoán thì dòng vốn đầu tư từ nay c...
...,...,...,...
3039,not_entailment,Ổ SSD dùng để khai thác tiền số Chia chỉ còn t...,{Ổ cứng đã được sử dụng để khai thác tiền số c...
3040,not_entailment,"Khi khai thác Chia, những ổ cứng dung lượng lớ...",{Tuổi thọ trung bình của những ổ cứng sử dụng ...
3041,not_entailment,Ổ cứng thường được hãng bảo hành từ ba đến năm...,{Ổ cứng được bảo hành tại các cửa hàng đại lý ...
3042,not_entailment,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...","{Ổ HDD được nhiều ""thợ đào"" Chia săn lùng vì t..."


In [16]:
dataset_not_entail

,gold_label,sentence1,sentence2
0,not_entailment,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",{Nhà máy Điện gió Trung Nam có 100% vốn cổ phầ...
1,not_entailment,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",{Nhiều người mong muốn được giữ quyền điều hàn...
2,not_entailment,"Cách đây một tháng, tập đoàn này cũng bán 49% ...",{Vốn của một công ty nước ngoài đang chiếm 49%...
3,not_entailment,"Dự án Nhà máy Điện gió Trung Nam, tổng số vốn ...",{Sản lượng điện dự kiến tạo ra trong một năm t...
4,not_entailment,"Từ nay tới năm 2028, dòng vốn đầu tư được dự b...",{Theo như dự đoán thì dòng vốn đầu tư từ nay c...
...,...,...,...
3039,not_entailment,Ổ SSD dùng để khai thác tiền số Chia chỉ còn t...,{Ổ cứng đã được sử dụng để khai thác tiền số c...
3040,not_entailment,"Khi khai thác Chia, những ổ cứng dung lượng lớ...",{Tuổi thọ trung bình của những ổ cứng sử dụng ...
3041,not_entailment,Ổ cứng thường được hãng bảo hành từ ba đến năm...,{Ổ cứng được bảo hành tại các cửa hàng đại lý ...
3042,not_entailment,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...","{Ổ HDD được nhiều ""thợ đào"" Chia săn lùng vì t..."


In [17]:
new_dataset_entail

,gold_label,sentence1,sentence2
0,entailment,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",Công ty Hitachi Sustainable Energy có cổ phần ...
1,entailment,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",Tập đoàn Trung Nam vừa thực hiện dự án điện nă...
2,entailment,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",Tập đoàn Trung Nam nắm giữ phần lớn lượng cổ p...
3,entailment,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",Tập đoàn Trung Nam là người có thể đưa ra quyế...
4,entailment,"Cách đây một tháng, tập đoàn này cũng bán 49% ...",Số cổ phần của tập đoàn Trung Nam tại Nhà máy ...
...,...,...,...
6089,entailment,Ổ cứng thường được hãng bảo hành từ ba đến năm...,Năm năm là mức thời gian bảo hành tối đa cho m...
6090,entailment,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...",Ổ SSD có tốc độ xử lý cao hơn ổ HDD.
6091,entailment,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...",Nhiều người dùng ổ cứng SSD để khai thác tiền ...
6092,entailment,"Chia là tiền điện tử đang ""sốt"" tại Trung Quốc.",Chia là một loại tiền ảo đang được rất nhiều n...


In [18]:
temp = []
for index1 in tqdm(range(len(new_dataset_entail))):
    # print(new_dataset_entail.iloc[index1]['sentence1'])
    for index2 in range(len(dataset_not_entail)):
        if new_dataset_entail.iloc[index1]['sentence1'] == dataset_not_entail.iloc[index2]['sentence1']:
            temp.append(dataset_not_entail.iloc[index2]['sentence2'])
            break

  0%|          | 0/6094 [00:00<?, ?it/s]

100%|██████████| 6094/6094 [09:18<00:00, 10.90it/s]


In [19]:
print(len(temp))

6094


In [20]:
new_dataset_entail['negative'] = temp
new_dataset_entail

,gold_label,sentence1,sentence2,negative
0,entailment,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",Công ty Hitachi Sustainable Energy có cổ phần ...,{Nhà máy Điện gió Trung Nam có 100% vốn cổ phầ...
1,entailment,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",Tập đoàn Trung Nam vừa thực hiện dự án điện nă...,{Nhà máy Điện gió Trung Nam có 100% vốn cổ phầ...
2,entailment,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",Tập đoàn Trung Nam nắm giữ phần lớn lượng cổ p...,{Nhiều người mong muốn được giữ quyền điều hàn...
3,entailment,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",Tập đoàn Trung Nam là người có thể đưa ra quyế...,{Nhiều người mong muốn được giữ quyền điều hàn...
4,entailment,"Cách đây một tháng, tập đoàn này cũng bán 49% ...",Số cổ phần của tập đoàn Trung Nam tại Nhà máy ...,{Vốn của một công ty nước ngoài đang chiếm 49%...
...,...,...,...,...
6089,entailment,Ổ cứng thường được hãng bảo hành từ ba đến năm...,Năm năm là mức thời gian bảo hành tối đa cho m...,{Ổ cứng được bảo hành tại các cửa hàng đại lý ...
6090,entailment,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...",Ổ SSD có tốc độ xử lý cao hơn ổ HDD.,"{Ổ HDD được nhiều ""thợ đào"" Chia săn lùng vì t..."
6091,entailment,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...",Nhiều người dùng ổ cứng SSD để khai thác tiền ...,"{Ổ HDD được nhiều ""thợ đào"" Chia săn lùng vì t..."
6092,entailment,"Chia là tiền điện tử đang ""sốt"" tại Trung Quốc.",Chia là một loại tiền ảo đang được rất nhiều n...,{Đồng tiền kém sức hút nhất tại Trung Quốc đượ...


In [21]:
dataset = Dataset.from_pandas(new_dataset_entail)
dataset = dataset.rename_column("sentence1", "anchor")
dataset = dataset.rename_column("sentence2", "positive")
dataset

Dataset({
    features: ['gold_label', 'anchor', 'positive', 'negative'],
    num_rows: 6094
})

In [23]:
def convert_dataset(example):
    temp = []
    for i in example['negative']:
        temp.append(
            {
                'gold_label': example['gold_label'], 
                'anchor': example['anchor'], 
                'positive': example['positive'], 
                'negative': i
            }
        )
    negative = 
    return temp
        
dataset = dataset.map(lambda example: convert_dataset(example), remove_columns=['gold_label', 'anchor', 'positive', 'negative'])
#     convert_dataset, 
#     remove_columns=['gold_label', 'anchor', 'positive', 'negative'],
#     batched=True
# )

Map:   0%|          | 0/6094 [00:00<?, ? examples/s]

TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'list'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.

In [58]:
dataset

Dataset({
    features: [],
    num_rows: 0
})